<b>Getting the Imports</b>

In [3]:
import tensorflow.keras.applications
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

<b>Downloading the VGG16 model</b>

In [7]:
vgg16 = tensorflow.keras.applications.vgg16.VGG16()

553467904/553467096 [==============================] - 1022s 2us/step


In [8]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

<b>Saving the model to disk</b>

In [4]:
vgg16.save("vgg16_model.h5")

NameError: name 'vgg16' is not defined

<b>Loading the model from disk</b>

In [5]:
vgg16 = load_model('vgg16_model.h5')

<b>Conveting to a Seqential</b><br></br>
<h>VGG16 is a 'Model' class object, we convert it into a 'Sequential' class object</h>

In [6]:
model = Sequential()
for layer in vgg16.layers:
    model.add(layer)

<b>Tunning the model so it predicts only on 2 classes</b>

In [ ]:
#Removing the last output layer
model.pop()

In [7]:
#Freezing rest of the layers 
for layer in model.layers:
    layer.trainable = False

#Adding the new output layer
output_layer = Dense(2, activation='softmax')
model.add(output_layer)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

<b>Saving the Fine Tunned Model</b>

In [29]:
model.save("finetunnedvgg16.h5")

<b>Loading the Fine Tunned Model</b>

In [9]:
model = load_model("finetunnedvgg16.h5")
print("Compiling Model...")
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
print("Model Complied!")

Compiling Model...
Model Complied!


<b>Training the new Output layer</b>

In [10]:
train_path = 'train'
valid_path = 'valid'
test_path = 'test'

classes = ['dog', 'cat']

In [11]:
generator = ImageDataGenerator(
                            rotation_range=15, 
                            horizontal_flip=True,
                            #vertical_flip=True, 
                            width_shift_range=0.1, 
                            height_shift_range=0.1, 
                            zoom_range=0.2,
                            channel_shift_range=50)

In [13]:
train_batches = generator.flow_from_directory(train_path, target_size=(224, 244), classes=classes, batch_size=5)
valid_batches = generator.flow_from_directory(valid_path, target_size=(224, 244), classes=classes, batch_size=2)
test_batches = generator.flow_from_directory(test_path, target_size=(224, 244), classes=classes, batch_size=2)

Found 100 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [15]:
model.fit_generator(train_batches, steps_per_epoch=20, epochs=10,
                   validation_data=valid_batches, validation_steps=5, verbose=2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 20 steps, validate for 5 steps
Epoch 1/10
20/20 - 30s - loss: 0.0449 - accuracy: 0.9800 - val_loss: 5.9596e-04 - val_accuracy: 1.0000
Epoch 2/10
20/20 - 33s - loss: 0.0495 - accuracy: 0.9800 - val_loss: 0.2207 - val_accuracy: 0.9000
Epoch 3/10
20/20 - 33s - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.8507 - val_accuracy: 0.9000
Epoch 4/10
20/20 - 32s - loss: 0.0299 - accuracy: 0.9800 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 5/10
20/20 - 33s - loss: 0.0400 - accuracy: 0.9900 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 6/10
20/20 - 32s - loss: 0.0409 - accuracy: 0.9800 - val_loss: 0.0862 - val_accuracy: 0.9000
Epoch 7/10
20/20 - 34s - loss: 0.0022 - accuracy: 1.0000 - val_loss: 1.6987e-04 - val_accuracy: 1.0000
Epoch 8/10
20/20 - 33s - loss: 0.0279 - accuracy: 0.9900 - val_loss: 4.1395e-05 - val_accuracy: 1.0000
Epoch 9/10
20/20 - 33s - loss: 0.0250 - accuracy: 0.9800 - val_loss: 9.9521e-05 - val_accuracy: 1.0000
Ep

<b>Saving the Trained Fine Tunned Model</b>

In [16]:
model.save('finetunned_trained.h5')